# 3. Merging 


This code is used to scrape and process data related to UFC fights, for the eventual purpose of creating a dataset for machine learning algorithms to predict the outcome of UFC fights.

Here's how it works, broken down into sections:

1. **Importing necessary libraries**: All required packages for this program are imported, such as pandas, numpy, matplotlib, seaborn, os etc. These libraries are essential for data processing and analysis, web scraping, data visualization, file handling, and more.

2. Changing the current directory to where all the relevant files and scripts are present.

3. **Loading the data files**: It loads .csv files located in the "fight_totals3" and "sig_strikes3" subfolders. These files presumably contain statistics for each fight and significant strikes during each fight respectively. A data frame is created for each of these (fight_totals and sig_strike_agg), with an additional 'fight_id' column added to identify the specific fight.

4. **Data Cleaning**: Each data frame is subset to remove unnecessary columns. The code checks for common columns present in both data frames and removes unnecessary columns. It also checks for missing values and takes action accordingly (either dropping the row or filling missing values). It replaces infinity values with zero.

5. **Merge the dataframes**: The two dataframes are then merged based on the common columns to form a new dataframe "fight_DF".

6. **Saving the dataframes**: Interim and final dataframes are saved to .csv files for future use.

The outputs of this code are stored dataframes containing cleaned and merged data on UFC fights, ready for further analysis or machine learning algorithms.

This script also seems to use Selenium and BeautifulSoup for web scraping purpose but does not use these in the provided code section. It might be used in another section which is not displayed here.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     
import shutil       
import datetime
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
import requests
import json
from random import randint
import  random
import os
os.chdir('C:/Users/Travis/OneDrive/Data Science/Personal_Projects/Sports/UFC_Prediction_V2')
from cmath import nan
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
import pickle
from sklearn.metrics import fbeta_score
import winsound
from bs4 import BeautifulSoup
import time

In [2]:
# Make master fight_totals dataframe
# load fight_totals files and sig_strike files, both from ufcstats.com
fight_totals_files = os.listdir('data/ufc_stats/fight_totals3')
sig_strike_files = os.listdir('data/ufc_stats/sig_strikes3')


In [3]:
# Make master fight_totals dataframe
fight_totals = pd.DataFrame()

for file in fight_totals_files: 
    file2 = file.replace('.csv','')
    df = pd.read_csv('data/ufc_stats/fight_totals3/'+file)
    # add fight id column
    df['fight_id'] = file2[:-7]
    fight_totals = fight_totals.append(df)
fight_totals = fight_totals.reset_index(drop=True)

fight_totals

,Unnamed: 0,Fighter_A,Fighter_B,A_Kd,B_Kd,A_Sig_strike_land,A_Sig_strike_att,B_Sig_strike_land,B_Sig_strike_att,A_Sig_strike_percent,...,B_Ctrl_time_min,B_Ctrl_time_sec,A_Ctrl_time_tot,B_Ctrl_time_tot,details,event_title,event_url,date,Winner,fight_id
0,0,Holly Holm,Irene Aldana,0,0,154,301,69,185,0.51,...,0,3,316,3,Mike Bell ...,UFC Fight Night: Holm vs. Aldana,http://www.ufcstats.com/event-details/805ad180...,"October 03, 2020","Holly Holm ""The Preacher's Daughter""",0005e00b07cee542
1,0,Paddy Pimblett,Kazula Vargas,0,0,3,6,7,9,0.50,...,2,43,53,163,Rear Naked Choke,UFC Fight Night: Volkov vs. Aspinall,http://www.ufcstats.com/event-details/1d007568...,"March 19, 2022","Paddy Pimblett ""The Baddy""",001441f70c293931
2,0,Greg Hardy,Ben Sosoli,0,0,54,105,26,97,0.51,...,0,0,0,0,Illegal Inhaler Use by Hardy La...,UFC Fight Night: Reyes vs. Weidman,http://www.ufcstats.com/event-details/3ae10ac4...,"October 18, 2019","Ben Sosoli ""Combat Wombat""",0019ec81fd706ade
3,0,Jared Rosholt,Josh Copeland,0,0,22,45,9,36,0.48,...,0,0,319,0,Punches to Head From Side Control,UFC 185: Pettis vs Dos Anjos,http://www.ufcstats.com/event-details/f54200f1...,"March 14, 2015",Jared Rosholt,0027e179b743c86c
4,0,Alistair Overeem,Stefan Struve,0,0,17,25,2,5,0.68,...,0,0,160,0,Punch to Head On Ground,UFC on FOX: Dos Santos vs Miocic,http://www.ufcstats.com/event-details/d4a12dfa...,"December 13, 2014","Alistair Overeem ""The Demolition Man""",002921976d27b7da
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5770,0,Jacare Souza,Francis Carmont,0,0,16,47,19,71,0.34,...,0,14,521,14,Richard Bertrand ...,UFC Fight Night: Machida vs Mousasi,http://www.ufcstats.com/event-details/51b0bb73...,"February 15, 2014",Jacare Souza,ffbc12e4f821ec68
5771,0,Darren Elkins,Lucas Martins,0,0,47,154,26,82,0.30,...,0,26,483,26,Richard Bertrand ...,UFC 179: Aldo vs Mendes 2,http://www.ufcstats.com/event-details/53f02bbc...,"October 25, 2014","Darren Elkins ""The Damage""",ffd3e3d37cba32da
5772,0,Sean Strickland,Krzysztof Jotko,0,0,84,182,37,196,0.46,...,0,0,0,0,Sal D'amato ...,UFC Fight Night: Reyes vs. Prochazka,http://www.ufcstats.com/event-details/d57e6a89...,"May 01, 2021",Sean Strickland,ffd8e8575a551875
5773,0,Tony Ferguson,Gleison Tibau,0,0,11,38,2,20,0.28,...,0,0,45,0,Rear Naked Choke,UFC 184: Rousey vs Zingano,http://www.ufcstats.com/event-details/ad4e9055...,"February 28, 2015","Tony Ferguson ""El Cucuy""",ffea776913451b6d


In [4]:
sig_strike_agg = pd.DataFrame()
for file in sig_strike_files:
    file2 = file.replace('.csv','')
    df = pd.read_csv('data/ufc_stats/sig_strikes3/'+file)
    df['fight_id'] = file2[:-11]
    sig_strike_agg = sig_strike_agg.append(df)
sig_strike_agg = sig_strike_agg.reset_index(drop=True)

sig_strike_agg

,Unnamed: 0,Fighter_A,Fighter_B,A_Head_Strikes_land,A_Head_Strikes_att,B_Head_Strikes_land,B_Head_Strikes_att,A_Head_Strikes_percent,B_Head_Strikes_percent,A_Body_Strikes_land,...,A_Ground_Strikes_land,A_Ground_Strikes_att,B_Ground_Strikes_land,B_Ground_Strikes_att,A_Ground_Strikes_percent,B_Ground_Strikes_percent,details,event_title,event_url,fight_id
0,0,Holly Holm,Irene Aldana,81,214,38,128,0.378505,0.296875,56,...,13,28,0,0,0.464286,NaN,Mike Bell ...,UFC Fight Night: Holm vs. Aldana,http://www.ufcstats.com/event-details/805ad180...,0005e00b07cee542
1,0,Paddy Pimblett,Kazula Vargas,1,4,5,7,0.250000,0.714286,0,...,1,1,3,3,1.000000,1.000000,Rear Naked Choke,UFC Fight Night: Volkov vs. Aspinall,http://www.ufcstats.com/event-details/1d007568...,001441f70c293931
2,0,Greg Hardy,Ben Sosoli,22,72,15,78,0.305556,0.192308,5,...,0,0,0,0,NaN,NaN,Illegal Inhaler Use by Hardy La...,UFC Fight Night: Reyes vs. Weidman,http://www.ufcstats.com/event-details/3ae10ac4...,0019ec81fd706ade
3,0,Jared Rosholt,Josh Copeland,19,41,7,34,0.463415,0.205882,3,...,10,17,0,0,0.588235,NaN,Punches to Head From Side Control,UFC 185: Pettis vs Dos Anjos,http://www.ufcstats.com/event-details/f54200f1...,0027e179b743c86c
4,0,Alistair Overeem,Stefan Struve,12,18,0,3,0.666667,0.000000,5,...,14,20,0,0,0.700000,NaN,Punch to Head On Ground,UFC on FOX: Dos Santos vs Miocic,http://www.ufcstats.com/event-details/d4a12dfa...,002921976d27b7da
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5769,0,Jacare Souza,Francis Carmont,10,39,14,59,0.256410,0.237288,4,...,6,9,0,0,0.666667,NaN,Richard Bertrand ...,UFC Fight Night: Machida vs Mousasi,http://www.ufcstats.com/event-details/51b0bb73...,ffbc12e4f821ec68
5770,0,Darren Elkins,Lucas Martins,25,113,16,70,0.221239,0.228571,17,...,1,2,0,0,0.500000,NaN,Richard Bertrand ...,UFC 179: Aldo vs Mendes 2,http://www.ufcstats.com/event-details/53f02bbc...,ffd3e3d37cba32da
5771,0,Sean Strickland,Krzysztof Jotko,46,130,17,163,0.353846,0.104294,24,...,0,0,0,0,NaN,NaN,Sal D'amato ...,UFC Fight Night: Reyes vs. Prochazka,http://www.ufcstats.com/event-details/d57e6a89...,ffd8e8575a551875
5772,0,Tony Ferguson,Gleison Tibau,5,27,2,18,0.185185,0.111111,3,...,3,8,0,0,0.375000,NaN,Rear Naked Choke,UFC 184: Rousey vs Zingano,http://www.ufcstats.com/event-details/ad4e9055...,ffea776913451b6d


In [5]:
fight_totals.to_csv('data/final/aggregates/All_Fight_Totals.csv', index=False)
sig_strike_agg.to_csv('data/final/aggregates/All_Sig_Strikes.csv', index=False)

In [6]:
# get the columns that are in both dataframes
common_cols = list(set(fight_totals.columns).intersection(sig_strike_agg.columns))
common_cols

['Fighter_B',
 'event_url',
 'Unnamed: 0',
 'fight_id',
 'Fighter_A',
 'event_title',
 'details']

In [7]:
# drop unnamed
fight_totals = fight_totals.drop(columns=['Unnamed: 0'])
sig_strike_agg = sig_strike_agg.drop(columns=['Unnamed: 0'])

In [8]:
# drop details from sig_strike_agg
sig_strike_agg = sig_strike_agg.drop(columns=['details'])

In [9]:
common_cols = list(set(fight_totals.columns).intersection(sig_strike_agg.columns))
common_cols

['event_url', 'Fighter_B', 'fight_id', 'Fighter_A', 'event_title']

In [10]:
# merge the two dataframes, using the common columns as the key
fight_DF = pd.merge(fight_totals, sig_strike_agg, on=common_cols, how= 'inner')
fight_DF

,Fighter_A,Fighter_B,A_Kd,B_Kd,A_Sig_strike_land,A_Sig_strike_att,B_Sig_strike_land,B_Sig_strike_att,A_Sig_strike_percent,B_Sig_strike_percent,...,B_Clinch_Strikes_land,B_Clinch_Strikes_att,A_Clinch_Strikes_percent,B_Clinch_Strikes_percent,A_Ground_Strikes_land,A_Ground_Strikes_att,B_Ground_Strikes_land,B_Ground_Strikes_att,A_Ground_Strikes_percent,B_Ground_Strikes_percent
0,Holly Holm,Irene Aldana,0,0,154,301,69,185,0.51,0.37,...,4,4,1.000000,1.000000,13,28,0,0,0.464286,NaN
1,Paddy Pimblett,Kazula Vargas,0,0,3,6,7,9,0.50,0.77,...,2,2,0.000000,1.000000,1,1,3,3,1.000000,1.000000
2,Greg Hardy,Ben Sosoli,0,0,54,105,26,97,0.51,0.26,...,0,1,1.000000,0.000000,0,0,0,0,NaN,NaN
3,Jared Rosholt,Josh Copeland,0,0,22,45,9,36,0.48,0.25,...,3,8,0.750000,0.375000,10,17,0,0,0.588235,NaN
4,Alistair Overeem,Stefan Struve,0,0,17,25,2,5,0.68,0.40,...,0,1,1.000000,0.000000,14,20,0,0,0.700000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5769,Jacare Souza,Francis Carmont,0,0,16,47,19,71,0.34,0.26,...,0,1,1.000000,0.000000,6,9,0,0,0.666667,NaN
5770,Darren Elkins,Lucas Martins,0,0,47,154,26,82,0.30,0.31,...,7,11,0.480769,0.636364,1,2,0,0,0.500000,NaN
5771,Sean Strickland,Krzysztof Jotko,0,0,84,182,37,196,0.46,0.18,...,0,0,NaN,NaN,0,0,0,0,NaN,NaN
5772,Tony Ferguson,Gleison Tibau,0,0,11,38,2,20,0.28,0.10,...,0,0,NaN,NaN,3,8,0,0,0.375000,NaN


In [11]:
# check for nulls
fight_DF.isnull().sum()

Fighter_A                      0
Fighter_B                     20
A_Kd                           0
B_Kd                           0
A_Sig_strike_land              0
                            ... 
A_Ground_Strikes_att           0
B_Ground_Strikes_land          0
B_Ground_Strikes_att           0
A_Ground_Strikes_percent    2040
B_Ground_Strikes_percent    2496
Length: 80, dtype: int64

In [12]:
# drop when Fighter_B is null
fight_DF = fight_DF.dropna(subset=['Fighter_B'])

In [13]:
missing = fight_DF.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing

A_Total_Strikes_percent         11
B_Total_Strikes_percent         15
details                         15
A_Distance_Strikes_percent      34
B_Distance_Strikes_percent      38
A_Head_Strikes_percent          39
B_Head_Strikes_percent          54
A_Body_Strikes_percent         493
B_Body_Strikes_percent         577
A_Leg_Strikes_percent          986
B_Leg_Strikes_percent          987
A_Clinch_Strikes_percent      1291
B_Clinch_Strikes_percent      1304
A_Takedown_percent            1831
A_Ground_Strikes_percent      2033
B_Takedown_percent            2110
B_Ground_Strikes_percent      2490
A_Sub_Success_Percent         4241
B_Sub_Success_Percent         4580
dtype: int64

In [14]:
# drop when Winner is missing
fight_DF = fight_DF.dropna(subset=['Winner'])

In [15]:
# change all the nulls to 0
fight_DF = fight_DF.fillna(0)

In [16]:
# change all the inf to 0
fight_DF = fight_DF.replace([np.inf, -np.inf], 0)

Save Fight_DF, the eventual dataset we train/test on.

In [17]:
fight_DF.to_csv('data/final/aggregates/Fight_DF.csv', index=False)